## Try to get all hydrogens around metal atoms about a conventional cell

In [1]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import defaultdict

## Count the amount of a specie in a cell

In [2]:
pmg_struct = Structure.from_file("./CaH6.cif")
pmg_struct.composition.to_data_dict
# {
#   'reduced_cell_composition': Comp: Li2 Mg1 H16,
#   'unit_cell_composition': defaultdict(float,
#              {'Li': 16.0, 'Mg': 8.0, 'H': 128.0}),
#   'reduced_cell_formula': 'Li2MgH16',
#   'elements': ['Li', 'Mg', 'H'],
#   'nelements': 3
# }
pmg_struct.composition.num_atoms            # get amounts of all atoms
pmg_struct.composition.alphabetical_formula # 'H128 Li16 Mg8'
pmg_struct.composition.anonymized_formula   # 'AB2C16'
pmg_struct.composition.iupac_formula        # 'Li16 Mg8 H128'
pmg_struct.composition.chemical_system      # 'H-Li-Mg'
pmg_struct.composition.elements             # [Element Li, Element Mg, Element H]
pmg_struct.composition.special_formulas     #{
                                            #     'LiO': 'Li2O2',
                                            #     'NaO': 'Na2O2',
                                            #     'KO': 'K2O2',
                                            #     'HO': 'H2O2',
                                            #     'CsO': 'Cs2O2',
                                            #     'RbO': 'Rb2O2',
                                            #     'O': 'O2',
                                            #     'N': 'N2',
                                            #     'F': 'F2',
                                            #     'Cl': 'Cl2',
                                            #     'H': 'H2'
                                            # }
pmg_struct.composition.get_el_amt_dict()                    # defaultdict(float, {'Li': 16.0, 'Mg': 8.0, 'H': 128.0})
pmg_struct.composition.get_integer_formula_and_factor()     # ('Li2MgH16', 8.0)
pmg_struct.composition.get_reduced_composition_and_factor() # (Comp: Li2 Mg1 H16, 8)


(Comp: Ca1 H6, 2)

## **一个小工具** 输入一个结构，输出其约化化学式，空间群对称性，文件名

In [3]:
import sys
from pathlib import Path

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


struct = Structure.from_file("../test/POSCAR")
name = struct.composition.get_integer_formula_and_factor()[0]
spg = SpacegroupAnalyzer(struct)
num = spg.get_space_group_number()
spg_symbol = spg.get_space_group_symbol()
print(f"{num}-{spg_symbol}-{name}")
struct

229-Im-3m-CaH6


Structure Summary
Lattice
    abc : 4.163148660818516 4.163148660818516 4.163148660818515
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 55.54485879776749
      A : -2.403595 2.403595 2.403595
      B : 2.403595 -2.403595 2.403595
      C : 2.403595 2.403595 -2.4035949999999997
    pbc : True True True
PeriodicSite: Ca (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: H (1.2018, -0.0000, 2.4036) [0.5000, 0.7500, 0.2500]
PeriodicSite: H (1.2018, 2.4036, -0.0000) [0.5000, 0.2500, 0.7500]
PeriodicSite: H (2.4036, 1.2018, -0.0000) [0.2500, 0.5000, 0.7500]
PeriodicSite: H (-0.0000, 1.2018, 2.4036) [0.7500, 0.5000, 0.2500]
PeriodicSite: H (0.0000, 2.4036, 1.2018) [0.7500, 0.2500, 0.5000]
PeriodicSite: H (2.4036, 0.0000, 1.2018) [0.2500, 0.7500, 0.5000]

In [4]:
import numpy as np


r_0, r_180, r_90, r_120 = 0, 0, 0, 0
pgops = spg.get_point_group_operations()

for idx, op in enumerate(pgops):
    # print(op.as_xyz_str())
    
    rot = op.rotation_matrix
    trc = np.trace(rot)
    det = np.linalg.det(rot) 
    if det > 0: # det(rot)=1: rotation; 
        trc = np.trace(rot)
        theta = np.degrees(np.arccos((trc-1)/2))
    else: # det(rot)=-1: rotoinversion
        trc = -np.trace(rot)
        theta = np.degrees(np.arccos((trc-1)/2))
    print("---------- {} ----------".format(idx+1))
    print(rot)
    print("trc={:>4.1f}  det={:>4.1f}  theta={:>4.1f}".format(trc, det, theta))

    if np.allclose(theta, 0.0):
        r_0 += 1
    elif np.allclose(theta, 180.0):
        r_180 += 1
    elif np.allclose(theta, 90.0):
        r_90 += 1
    elif np.allclose(theta, 120.0):
        r_120 += 1

    for idx1, site1 in enumerate(struct):
        # print("----------check {} ----------".format(site1.label+str(idx1)))
        for idx2, site2 in enumerate(struct):
            res = op.are_symmetrically_related(site1.coords, site2.coords)
            if res:
                # print((site1==site2) and (idx1==idx2), site1.label+str(idx1), site2.label+str(idx2))
                print(site1.label+str(idx1), site2.label+str(idx2))

# print("r_0={} r_180={} r_90={} r_120={}".format(r_0, r_180, r_90, r_120))


---------- 1 ----------
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
trc= 3.0  det= 1.0  theta= 0.0


AttributeError: label

### 计算共轭类

In [6]:
from typing import List
from collections import defaultdict
from pymatgen.symmetry.analyzer import SymmOp
from numpy import array


def classify(operations:List[SymmOp]):
    
    classified_ops = defaultdict(list)

    # 首先对对称操作按照旋转的角度和行列式值分类
    for idx, op in enumerate(operations):
        # print(op.as_xyz_str())
        rot = op.rotation_matrix
        # print(rot)
        trc = np.trace(rot)
        det = np.linalg.det(rot) 
        if det > 0: # det(rot)=1: rotation; 
            trc = np.trace(rot)
            theta = np.degrees(np.arccos((trc-1)/2))
            if np.allclose(theta, 0.0):
                classified_ops[f'identity'].append(op)
            else:
                classified_ops[f'rotation_{int(theta)}'].append(op)
        else: # det(rot)=-1: rotoinversion
            trc = -np.trace(rot)
            theta = np.degrees(np.arccos((trc-1)/2))
            if np.allclose(theta, 0.0):
                classified_ops[f'inversion'].append(op)
            else:
                classified_ops[f'rotoinversion_{int(theta)}'].append(op)
            
    # print(dict(classified_ops))
    # 进一步按照共轭类分类，因为有些二重轴不是一类。
    for typename, elements in classified_ops.items(): # 在这里，typename 的类型是 str，elements 的类型是 List[SymmOp]
        print(typename, "-------------------------------")
        # if typename == "inversed_rot_90":
            # break
        x: SymmOp = np.random.choice(elements)
        xm: array = x.rotation_matrix
        xm_i:array= np.linalg.inv(xm)
        am_s: array = [e.rotation_matrix for e in elements]
        bm_s: array = [np.dot(np.dot(xm_i, am), xm) for am in am_s]
        if typename == "rotation_180 ":
            print(am_s)
            print(bm_s)
        flag = True
        for bm in bm_s:
            for am in am_s:
                # print(np.allclose(am, bm))
                if np.allclose(am, bm): # 只要有一个am和bm一样, 就不再往下找了
                    flag = True
                    break 
            else: # 全部的am_s都走遍, 也没找到一个am=bm
                # print("dsfdsfsdf")
                flag = False
        print(flag)


    # return dict(classified_ops)

pgops = spg.get_point_group_operations()
classify(pgops)

identity -------------------------------
True
inversion -------------------------------
True
rotation_90 -------------------------------
True
rotoinversion_90 -------------------------------
True
rotation_180 -------------------------------
True
rotoinversion_180 -------------------------------
True
rotation_120 -------------------------------
True
rotoinversion_120 -------------------------------
True


In [17]:
def is_in_matrix(m:array,ms:List[array]):
    # 判断矩阵m是否在矩阵组成的列表ms中
    flag = True
    for _ in ms:
        if np.allclose(m, _):
            flag = True
            break
    else:
        flag = False
    return flag

def kill_duplicated_element(ms:array):
    # 在一组 3*3的矩阵中，找到重复的并删除
    new_ms = []
    for m in ms:
        if not new_ms:
            new_ms.append(m)
        else:
            if not is_in_matrix(m=m, ms=new_ms):
                new_ms.append(m)
    return new_ms

def is_equal_for_cless(cl1:List[array], cl2:List[array]):
    # 判断两个类是否相同
    flag = True
    for m1 in cl1:
        if is_in_matrix(m1, cl2):
            flag = True
            break
    else:
        flag = False
    return flag

def is_in_clesses(cl1:List[array], clesses:List[List[array]]):
    flag = True
    for cl2 in clesses:
        if is_equal_for_cless(cl1, cl2):
            flag = True
            break
    else:
        flag = False
    return flag

def kill_duplicated_clesses(clesses:array):
    new_clesses = []
    for cl in clesses: # cl此时是一个类，里面有很多个3*3的矩阵
        if not new_clesses:
            new_clesses.append(cl)
        else:
            if not is_in_clesses(cl1=cl, clesses=new_clesses):
                new_clesses.append(cl)
    return new_clesses

clesses = []
for idx, op in enumerate(pgops):
    # print(op.as_xyz_str())
    bm = [np.dot(np.dot(np.linalg.inv(x.rotation_matrix), op.rotation_matrix), x.rotation_matrix) for x in pgops]
    cless = kill_duplicated_element(bm)
    # print(cless)
    clesses.append(cless)
clesses = kill_duplicated_clesses(clesses)
for c in clesses:
    print(len(c))

1
1
6
6
3
3
6
6
8
8


In [ ]:
xm = np.array([[0.,  1.,  0.],
               [ 0.,  1., -1.],
               [-1.,  1.,  0.]])
xm_1=np.array([[ 1., -0., -1.],
               [ 1.,  0.,  0.],
               [ 1., -1.,  0.]])
am=np.array([[ 1., -1.,  0.],
             [ 1.,  0., -1.],
             [ 1.,  0.,  0.],])
# np.dot(xm_1, am)
np.dot(np.dot(xm_1, am), xm)

In [ ]:
from pyxtal.symmetry import OperationAnalyzer
from pyxtal import pyxtal
from monty.serialization import dumpfn

# structjson = struct.to_json("../test/CaH6.json")
dumpfn(struct, "../test/CaH6.json")
px = pyxtal().load_dict({
    "group":229,
    "dim":3,
    "lattice":[[-1.698432, 1.698432, 1.698432],[1.698432, -1.698432, 1.698432],[1.698432, 1.698432, -1.698432]],
    "factor":1.0,
    "numIons":[2, 12],
    "sites":[['2a'], ['12e']],
    "molecular":[],
    "source":[],
    "PBC":[1,1,1],
    "numMols":[],
    "valid":True,
    "formula":"CaH6"
})
# pgops = spg.get_point_group_operations()

# for idx, op in enumerate(pgops):
#     print(idx, OperationAnalyzer(op))

TypeError: list indices must be integers or slices, not str

## Voronoi模块的使用

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.analysis.chemenv.coordination_environments.structure_environments \
    import ChemicalEnvironments, StructureEnvironments, DetailedVoronoiContainer
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies \
    import SimplestChemenvStrategy

structure = Structure.from_file("./CaH6.cif")
vc = DetailedVoronoiContainer(structure)
# se = StructureEnvironments(voronoi_container)
# sc = SimplestChemenvStrategy()
# sc.get_site_neighbors(structure)

In [ ]:
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import NearNeighbors

s = Structure.from_file("./La2Y6H46_ContributedBy_Mayuan.vasp")
# graph = StructureGraph.with_local_env_strategy(structure, strategy=nn)

In [ ]:
from pymatgen.analysis.local_env import VoronoiNN

vnn = VoronoiNN()
all_vr = vnn.get_all_voronoi_polyhedra(structure)
all_vr

In [ ]:
from pymatgen.analysis.local_env import VoronoiNN

vnn = VoronoiNN()
all_vr = vnn.get_all_voronoi_polyhedra(structure)
all_vr

# 获得一个晶体晶胞的原子数

In [ ]:
from pymatgen.core.structure import Structure
pmg_struct = Structure.from_file("./CaH6.cif")
pmg_struct.density
pmg_struct.num_sites

# 替换元素

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.io.vasp import Poscar
# filenames1 = ["LaCeH8-200GPa-P4mmm", "LaCeH8-200GPa-Pmmn", "LaCeH18-200GPa-Amm2", 
#              "LaCeH20-200GPa-I41amd", "LaCeH20-200GPa-P-6m2", "LaCeH20-200GPa-P63mmc",
#             #  "LaCeH20-200GPa-R-3m",]
filenames2 = ["Y1Ce1H7-100GPa-Pmma", "YCeH5-100GPa-C2m", "YCeH8-100GPa-P4mmm", "YCeH18-200GPa-P-6m2", "YCeH20-400-P4mmm"]

for file in filenames2:
    s = Structure.from_file(file+".cif")
    s.replace_species({'Y':'Ce', 'Ce':'Sr'})
    formula=s.composition.reduced_formula.replace(' ', '')
    filename="f1-"+formula+'-'+'-'.join(file.split('-')[1:])
    print(filename)
    Poscar(s).write_file(filename=filename+".vasp")